In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from chromadb.utils.embedding_functions import OllamaEmbeddingFunction
import pandas as pd

embeddings = OllamaEmbeddings(model="mxbai-embed-large")
ef = OllamaEmbeddingFunction(
    model_name="nomic-embed-text",
    url="http://localhost:11434/api/embeddings",
)

books = pd.read_csv("data/cleaned.csv")
raw_documents = TextLoader("data/tag_descriptions_cleaned.csv").load()
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

books_db = Chroma.from_documents(documents=raw_documents, embedding=embeddings, persist_directory='./')

Created a chunk of size 2005, which is longer than the specified 2000
Created a chunk of size 2007, which is longer than the specified 2000
Created a chunk of size 2821, which is longer than the specified 2000
Created a chunk of size 2501, which is longer than the specified 2000
Created a chunk of size 2280, which is longer than the specified 2000
Created a chunk of size 2603, which is longer than the specified 2000
Created a chunk of size 2031, which is longer than the specified 2000
Created a chunk of size 2741, which is longer than the specified 2000
Created a chunk of size 2108, which is longer than the specified 2000
Created a chunk of size 2200, which is longer than the specified 2000
Created a chunk of size 2872, which is longer than the specified 2000
Created a chunk of size 2399, which is longer than the specified 2000
Created a chunk of size 2121, which is longer than the specified 2000
Created a chunk of size 2164, which is longer than the specified 2000
Created a chunk of s

In [ ]:
# querying
query = "Christie for Christmas"

# db_books.query(
#     query_texts=["doc10", "thus spake zarathustra", ...],
#     n_results=10,
#     where={"metadata_field": "is_equal_to_this"},
#     where_document={"$contains":"search_string"}
# )
results = books_db.similarity_search_with_score(query, k=5)


for res, score in results:
#    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")
    print(score)

0.9264920353889465
0.9264920353889465
0.9264920353889465
0.9264920353889465
0.9264920353889465
